# B. Tree-based Method for Anomaly Detection

Goal of this workbook

1. Know when to apply
2. What's the difference with a usual random forest
3. Know how to apply
4. Try a few detection methods with various data sets

In [95]:
# dataframe / analysis tools
import pandas as pd
import numpy as np
import scipy as sp
from scipy.io import loadmat
from sklearn.metrics import confusion_matrix, classification_report
from scipy.stats import multivariate_normal, uniform
# plotting
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

# Helpful functions
def normalize(x):
    mu = np.mean(x,axis=0)
    sigma = np.std(x,axis=0)
    return (x - mu)/sigma

def ad_plot(x, y, mask):
    plt.figure(figsize=(16,8))
    sns.kdeplot(x, y)
    sns.regplot(x=x[~mask], y=y[~mask], fit_reg=False,color='g',scatter_kws={'alpha':0.25})
    sns.regplot(x=x[mask], y=y[mask],fit_reg=False,color='red')
    
def estimate_gaussian(x):
    # Step 1: Normalize
    x = normalize(x)
    # Step 2: Use sp.stats.norm.pdf on results above
    x = x.apply(lambda v: sp.stats.norm.pdf(v),1)
    # Step 3: get Probabilities of Feature_1 x Features_2 x .... Feature_n
    return x.apply(np.prod,1)

def get_mu_sig(x):
    mu = np.mean(x, axis=0)
    sigma = np.cov(x.T)
    return mu, sigma

def multivariateGaussian(x,mu,sigma):
    p = multivariate_normal(mean=mu, cov=sigma)
    return p.pdf(x)

## Datasets
[Lymph]('https://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+%28Diagnostic%29')

Features are computed from a digitized image of a fine needle aspirate (FNA) of a breast mass. They describe characteristics of the cell nuclei present in the image. A few of the images can be found at [Web Link] 


[MUSK]('https://archive.ics.uci.edu/ml/datasets/Musk+%28Version+2%29')

This dataset describes a set of 102 molecules of which 39 are judged by human experts to be musks and the remaining 63 molecules are judged to be non-musks. The goal is to learn to predict whether new molecules will be musks or non-musks. However, the 166 features that describe these molecules depend upon the exact shape, or conformation, of the molecule. Because bonds can rotate, a single molecule can adopt many different shapes. To generate this data set, all the low-energy conformations of the molecules were generated to produce 6,598 conformations. Then, a feature vector was extracted that describes each conformation. 


[Women's Breast Cancer]('http://odds.cs.stonybrook.edu/lympho/')

*missing info*


In [221]:
df_musk = loadmat('./data/musk.mat')
df_wbc = loadmat('./data/wbc.mat')
df_lymph = loadmat('./data/lympho.mat')

## Your turn:
Using methods from before, apply these methods to the data. Look at the shape of distributions and correct to normal if needed. Use scatter or density plots to explore your data. 


Guiding Questions
1. Do these models work well? 
2. If no, why not?
3. Try working with an isolation forest model. Does it work well? Why or why not?

## Isolation Forest:
- Tree based method: Like Random forest, takes subsets of data. This means it's great for high-dimensional data.
- Not cluster or density based
- Works by randomly partitioning data set. 
- The more partitions, the more "normal" the data point is. The fewer, the more anomalous it is. 
- Get an "anomaly score". Essentially, the average of each point's numbers of splits to reach averaged across all trees and and ordered. 
- Whereas in Gaussian and Multivariate Gaussian, we can build relatively simple decision boundaries, with iForest you can build more complex ones

### Isolation Trees Make up an Isolation Forest
![title](http://pubs.rsc.org/services/images/RSCpubs.ePlatform.Service.FreeContent.ImageService.svc/ImageService/Articleimage/2016/AY/c6ay01574c/c6ay01574c-f1_hi-res.gif)


### In an iTree, how many nodes does it take to isolate?

![title](https://image.slidesharecdn.com/gerster-anomalyv4-150730232953-lva1-app6892/95/anomaly-detection-using-isolation-forests-10-638.jpg)

In [ ]:
# your turn -- choose a data set and try iso forest on it.